In [1]:
import requests
import pandas as pd
import time
import math

API_KEYS = [
    "468fd187-9460-42b3-adcb-934315fb3c60", 
    "f68ae405-a48a-432d-a4f8-1847ee3afd3c",
    "69969001-6dea-4fa0-8490-533cff7aabcf",
    "66715fc5-9def-486c-8cb7-c6c943833fd1",
    "4614076f-20c1-4632-941c-8a47dec5d386",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "06b3535f-186f-4cb7-be6d-14ae08e4b48b"
]

SAVE_INTERVAL = 150  # save every 150 processed rows


def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(API_KEYS)

    # Auto batching for API key rotation
    batch_size = math.ceil(total_rows / total_keys)

    print(f"Total rows: {total_rows}")
    print(f"Total API keys: {total_keys}")
    print(f"Auto batch size = {batch_size}\n")

    save_counter = 0  # count rows for autosave

    for i in range(total_rows):
        # Pick API key by row index
        key_index = min(i // batch_size, total_keys - 1)
        current_api_key = API_KEYS[key_index]

        phone_number = df.loc[i, "Whatsapp"]
        result = check_number_status(phone_number, current_api_key)

        df.loc[i, "validity"] = result

        print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | {phone_number} → {result}")

        time.sleep(5)
        save_counter += 1

        # Save file every 150 processed rows
        if save_counter >= SAVE_INTERVAL:
            df.to_excel("autosave_result.xlsx", index=False)
            print("💾 Autosaved last 150 rows → autosave_result.xlsx")
            save_counter = 0

    # Final save after finishing all rows
    df.to_excel("final_result.xlsx", index=False)
    print("\n🎉 Done! Saved final_result.xlsx")


# Example
# df = pd.read_excel("input.xlsx")
# process_dataframe(df)


In [2]:
# load data
database = pd.read_excel("Database.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")

In [3]:
no_crm = database[database['CRM'].isna()]
uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [4]:
uncheck

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,45244,6282157218899,Muhammad Bahrun Emni,November,2025,NaN,Web Ads,100552,1,SUDAN,1,05-06,100552,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
1,45241,6282143044358,Lilik Saputra,November,2025,NaN,Web Ads,100063,1,PALESTINA,1,21-22,100063,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,NaN
2,45115,6285145152768,Almi tatar,November,2025,NaN,Web Ads,50853,1,SUDAN,1,13-14,50853,Fri,Jumat,2025,Aktif,BRONZE,< 100.000,NaN
3,45114,628128595476,Frima Derris,November,2025,NaN,Web Ads,304171,1,PALESTINA,1,13-14,304171,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,NaN
4,45113,6287726687866,nana ginanjar,November,2025,NaN,Web Ads,104158,1,PALESTINA,1,10-11,104158,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,45173,6281269445910,Ich,November,2025,NaN,Web Ads,103572,1,SUDAN,1,00-01,103572,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN
132,45172,6287770973744,Jamarullah,November,2025,NaN,Web Ads,103685,1,PALESTINA,1,21-22,103685,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,NaN
133,45171,628127519259,DEDE FIRMANSYAH,November,2025,NaN,Web Ads,50024,1,SUDAN,1,16-17,50024,Fri,Jumat,2025,Aktif,BRONZE,< 100.000,NaN
134,45181,62817720530,Yuni Anshori,November,2025,NaN,Web Ads,100357,1,SUDAN,1,04-05,100357,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,NaN


In [5]:
process_dataframe(uncheck)

Total rows: 136
Total API keys: 8
Auto batch size = 17

Row 1/136 | API 1/8 | 6282157218899 → Number registered
Row 2/136 | API 1/8 | 6282143044358 → Number registered
Row 3/136 | API 1/8 | 6285145152768 → Number registered
Row 4/136 | API 1/8 | 628128595476 → Number registered
Row 5/136 | API 1/8 | 6287726687866 → Number registered
Row 6/136 | API 1/8 | 6282211332977 → Number registered
Row 7/136 | API 1/8 | 6281278685420 → Number registered
Row 8/136 | API 1/8 | 628118542600 → Number registered
Row 9/136 | API 1/8 | 6283869655110 → Number registered
Row 10/136 | API 1/8 | 6281378246658 → Number registered
Row 11/136 | API 1/8 | 6281317522334 → Number registered
Row 12/136 | API 1/8 | 6281315701979 → Number registered
Row 13/136 | API 1/8 | 6287834021000 → Number registered
Row 14/136 | API 1/8 | 6281256545271 → Number registered
Row 15/136 | API 1/8 | 6285384632337 → Number registered
Row 16/136 | API 1/8 | 628125120052 → Number registered
Row 17/136 | API 1/8 | 62895320425374 → Numb

In [6]:
# process_dataframe(uncheck)
val_1 = pd.read_excel("final_result.xlsx")
# val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

In [7]:
validity = pd.concat([val_1])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,45244,6282157218899,Muhammad Bahrun Emni,November,2025,NaN,Web Ads,100552,1,SUDAN,1,05-06,100552,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
1,45241,6282143044358,Lilik Saputra,November,2025,NaN,Web Ads,100063,1,PALESTINA,1,21-22,100063,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
2,45115,6285145152768,Almi tatar,November,2025,NaN,Web Ads,50853,1,SUDAN,1,13-14,50853,Fri,Jumat,2025,Aktif,BRONZE,< 100.000,Number registered
3,45114,628128595476,Frima Derris,November,2025,NaN,Web Ads,304171,1,PALESTINA,1,13-14,304171,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
4,45113,6287726687866,nana ginanjar,November,2025,NaN,Web Ads,104158,1,PALESTINA,1,10-11,104158,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,45173,6281269445910,Ich,November,2025,NaN,Web Ads,103572,1,SUDAN,1,00-01,103572,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
132,45172,6287770973744,Jamarullah,November,2025,NaN,Web Ads,103685,1,PALESTINA,1,21-22,103685,Fri,Jumat,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
133,45171,628127519259,DEDE FIRMANSYAH,November,2025,NaN,Web Ads,50024,1,SUDAN,1,16-17,50024,Fri,Jumat,2025,Aktif,BRONZE,< 100.000,Number registered
134,45181,62817720530,Yuni Anshori,November,2025,NaN,Web Ads,100357,1,SUDAN,1,04-05,100357,Sat,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered


In [8]:
validity_update = pd.concat([data_with_validity, validity])
validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [11]:
validity_update['validity'].value_counts(dropna=False)

validity
NaN                                                                                                                                                21533
Number registered                                                                                                                                  19077
Number not registered                                                                                                                               2608
Error: 504 Server Error: Gateway Time-out for url: https://api.starsender.online/api/check-number                                                     14
Error: 502 Server Error: Bad Gateway for url: https://api.starsender.online/api/check-number                                                           4
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))        3
Timeout                                                                  

In [12]:
validity_update = validity_update[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM', 'Source',
       'Total', 'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori', 'validity']]

In [13]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")